In [1]:
# import the needed libraries.
import pandas
import researchpy as rp
import seaborn as sns
import numpy as np
import pandas as pd

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp

In [2]:
df = pd.read_csv('results.csv', sep = '\t')
df = df.set_index('Unnamed: 0')
# remove the name of Indexing.
df.index.name = None
#make the accuracy * 100.
df['final_accuracy'] = df['final_accuracy']*100
df


,a,b,Q_bar,final_accuracy
0,1,1,0.1,94.585000
1,1,1,0.1,97.488999
2,1,1,0.1,97.108001
3,1,1,0.2,97.641001
4,1,1,0.2,97.595000
5,1,1,0.2,95.202000
6,1,2,0.1,97.412000
7,1,2,0.1,98.050000
8,1,2,0.1,97.633000
9,1,2,0.2,95.882000


In [3]:
rp.summary_cont(df.groupby(['a', 'b','Q_bar']))['final_accuracy']
#Sd = standard deviation, SE standard error.

N     Mean      SD      SE  95% Conf.  Interval
a b Q_bar                                                 
1 1 0.1    3  96.3940  1.5782  0.9112    92.4736  100.3144
    0.2    3  96.8127  1.3951  0.8054    93.3471  100.2782
  2 0.1    3  97.6983  0.3240  0.1870    96.8935   98.5031
    0.2    3  95.7293  2.5015  1.4442    89.5153  101.9434
2 1 0.1    3  96.6163  0.8675  0.5009    94.4612   98.7714
    0.2    3  94.2550  5.4483  3.1456    80.7206  107.7894
  2 0.1    3  97.3123  0.3878  0.2239    96.3489   98.2757
    0.2    3  95.7073  2.9814  1.7213    88.3012  103.1134

In [4]:
# Running Ols to find the anova model afterwards.
model = ols('final_accuracy ~ C(a)*C(b)*C(Q_bar)', df).fit()

# Seeing if the overall model is significant
print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")

Overall model F( 7, 16) =  0.560, p =  0.7774


In [5]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         final_accuracy   R-squared:                       0.197
Model:                            OLS   Adj. R-squared:                 -0.155
Method:                 Least Squares   F-statistic:                    0.5598
Date:                Thu, 15 Oct 2020   Prob (F-statistic):              0.777
Time:                        19:00:53   Log-Likelihood:                -51.246
No. Observations:                  24   AIC:                             118.5
Df Residuals:                      16   BIC:                             127.9
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                              96.3940      1.447     66.601      0.000      93.326      99.462
C(a)[T.2]                               0.2223      2.047      0.109      0.915      -4.117       4.561
C(b)[T.2]                               1.3043      2.047      0.637      0.533      -3.035       5.643
C(Q_bar)[T.0.2]                         0.4187      2.047      0.205      0.841      -3.920       4.758
C(a)[T.2]:C(b)[T.2]                    -0.6083      2.895     -0.210      0.836      -6.745       5.528
C(a)[T.2]:C(Q_bar)[T.0.2]              -2.7800      2.895     -0.960      0.351      -8.916       3.356
C(b)[T.2]:C(Q_bar)[T.0.2]              -2.3877      2.895     -0.825      0.422      -8.524       3.749
C(a)[T.2]:C(b)[T.2]:C(Q_bar)[T.0.2]     3.1440      4.094      0.768      0.454      -5.534      11.822
==============================================================================
Omnibus:                        9.874   Durbin-Watson:                   2.497
Prob(Omnibus):                  0.007   Jarque-Bera (JB):                8.035
Skew:                          -1.097   Prob(JB):                       0.0180
Kurtosis:                       4.795   Cond. No.                         17.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
# Getting the final 3-way ANOVA table.
res = sm.stats.anova_lm(model, typ= 2)
res

,sum_sq,df,F,PR(>F)
C(a),2.822203,1.0,0.449083,0.512328
C(b),2.105154,1.0,0.334982,0.570798
C(Q_bar),11.412606,1.0,1.816030,0.196560
C(a):C(b),1.392981,1.0,0.221658,0.644134
C(a):C(Q_bar),2.188897,1.0,0.348308,0.563316
C(b):C(Q_bar),0.997969,1.0,0.158802,0.695530
C(a):C(b):C(Q_bar),3.706776,1.0,0.589841,0.453667
Residual,100.549909,16.0,NaN,NaN


In [7]:
Sum_Square_Error = res['sum_sq'].sum()
res['Percentage_Variation_Explained'] = (res['sum_sq'] / Sum_Square_Error) * 100
res

,sum_sq,df,F,PR(>F),Percentage_Variation_Explained
C(a),2.822203,1.0,0.449083,0.512328,2.254579
C(b),2.105154,1.0,0.334982,0.570798,1.681748
C(Q_bar),11.412606,1.0,1.816030,0.196560,9.117212
C(a):C(b),1.392981,1.0,0.221658,0.644134,1.112814
C(a):C(Q_bar),2.188897,1.0,0.348308,0.563316,1.748649
C(b):C(Q_bar),0.997969,1.0,0.158802,0.695530,0.797249
C(a):C(b):C(Q_bar),3.706776,1.0,0.589841,0.453667,2.961240
Residual,100.549909,16.0,NaN,NaN,80.326510
